In [31]:
import search
from csp import CSP, backtracking_search, min_conflicts, parse_neighbors, mrv, forward_checking
import random

# Homework 1

## 1.1

Talk about psychologists and AI for a couple paragraphs.

I think that introspection can give us insight into how we think. However, I think it would be extremely limited, as people are often not even sure of how they make their own decisions. I think that trying to model that same type of logic using AI would not be effective. You might be able to model a thought process, but some leaps that may seem random could actually come from someone's childhood background or some anecdote they once heard. Humans are not only driven by logic and reason, which makes it really hard to program. Which is most likely why the field of psychology veered away from introspection because it seemed scientifically unreliable.

In my into psych class, we discussed a man who had the emotions portion of his brain destroyed (I believe the frontal lobe), so that he was driven by logic. However, it affected his ability to make simple decisions like picking between a red and blue pen. While many would think he would be able to make better decisions as a completely rational individual, it actually made it more difficult, as he did not have preferences to help sway his decisions. In this same way, it seems that a program that deals in many absolutes would not be able to replicate the complexity of a single thought.


## 1.2

Describe your TSP problem formulation.

Your starting state would be visiting all the towns in sequential order, and returning to the initial town.

An action will be switching the order of two towns in order to try to optimize the solution and reduce the travel distance.


In [32]:
import random 
import time
import math
from search import Problem, hill_climbing, simulated_annealing, \
    exp_schedule, genetic_search

            
class Travelingsalesman(Problem):

    #Constructor has initial state and matrix of cities
    def __init__(self, initial, cities):
        self.initial = initial
        self.dist_cities = cities
        
    #Method actions returns list of 10 random possible actions   
    def actions(self, state):
        options = []
        
        for x in range(0,10):
            curr_path = state[:]
            #randomly switch two of the cities in the path
            t1 = random.randint(0, n - 1 )
            t2 = random.randint(0, n - 1 )
            t3 = curr_path[t1]
            curr_path[t1] = curr_path[t2]
            curr_path[t2] = t3
            #add new potential path to list of options
            options.append(curr_path)
        return options
    
    def result(self, stateIgnored, x):
        return x
    
    def value(self, state):
        result = 0
        # go through all the cities and back to original city
        for var in range(n):
            result += self.dist_cities[state[var]][state[((var+1) % n)]]    
        return result * (-1)
    
if __name__ == '__main__':
    list_cities = []

    for val in range(0, 10):
        list_cities.append(val)
    
    n = len(list_cities)
    dist_cities = [ [ 0 for i in range(n) ] for j in range(n) ]
    
    for town1 in range( 0, n):
        for town2 in range (0, n):
            if town1 == town2:
                dist_cities[town1][town1]= 0
            else:
                dist_cities[town1][town2] = random.randint(1,100)
                dist_cities[town2][town1] = random.randint(1,100)
     
        
    p = Travelingsalesman(list_cities, dist_cities)
    
    
    # Code from lab02 used to print out solutions and timing
    # Solve the problem using hill-climbing.
    time1 = time.time()
    hill_solution = hill_climbing(p)
    time2 = time.time()
    print('Hill-climbing solution       x: ' + str(hill_solution)
            + '\tvalue: ' + str(p.value(hill_solution))
            + "\t\ttime: %0.3f seconds" % (time2 - time1)
            )
    
    # Solve the problem using simulated annealing.
    time1 = time.time()
    annealing_solution = simulated_annealing(
        p,
        exp_schedule(k=20, lam=0.005, limit=1000)
    )
    time2 = time.time()
    print('Simulated annealing solution x: ' + str(annealing_solution)
            + '\tvalue: ' + str(p.value(annealing_solution))
            + "\t\ttime: %0.3f seconds" % (time2 - time1)
            )        
            

Hill-climbing solution       x: [4, 1, 2, 3, 0, 6, 9, 7, 8, 5]	value: -260		time: 0.001 seconds
Simulated annealing solution x: [9, 5, 1, 2, 4, 3, 0, 8, 7, 6]	value: -171		time: 0.081 seconds


The Simulated annealing solution works better because it essentially behaves in the same manner as hill-climbing, but has the initial capability to move to less ideal states when the 'temperature is high'. This allows the algorithm to less often become stuck in local maxima, as hill climbing does.


## 1.3

Describe your Course Scheduling Formulate

The variable is the course while the domain are the classroom, professor, time.

The Constraints are that a professor cannot be teaching two courses at once, and no classroom can be double booked during a specific time slot. And each class should be offered one time.

The Neighbors were modeled after the Zebra problem.

All these were used as the parameters to build a constraint satisfaction problem.
Below is the implementation.

This seemed like a good implementation style, as it made since for every class, that needs to be offered, be the variables.
While, the domains are all the numerous possibilities of professors, times and rooms.

In [33]:
courses = ['CS108', 'CS112', 'CS212', 'CS214', 'CS262', 'CS344']
profs = ['Norman', 'Adams', 'VanderLinden', 'Plantinga']
times =['8AM', '9AM', '10AM', '11AM']
rooms = ['NH064', 'SB382']

variables = courses
domain = []

# create list of possible combinations of 3 domains
for prof in profs:
    for time in times:
        for room in rooms:
            domain.append(prof + " " + time + " " + room)
            
domains = {}

# make a dictionary of each course with every possible domain
for course in courses:
    domains[course] = domain

# make neighbors modeled off of Zebra CPS
neighbors = parse_neighbors("""CS108: Norman 8AM""", variables)
for type in [courses, domain]:
        for A in type:
            for B in type:
                if A != B:
                    if B not in neighbors[A]:
                        neighbors[A].append(B)
                    if A not in neighbors[B]:
                        neighbors[B].append(A)


def class_constraints(A, a, B, b):
    a_prof = a.split()[0]
    a_room = a.split()[1]
    a_time = a.split()[2]

    b_prof = b.split()[0]
    b_room = b.split()[1]
    b_time = b.split()[2]
    
    #prof can't teach two classes at once
    if (a_prof == b_prof) and (a_time == b_time):
        return False
    #room cannot be double booked
    if (a_room == b_room) and (a_time == b_time):
        return False
    
    #Constaints to make certain professor teach certain courses
    if A == "CS212" and a_prof != "Plantinga":
        return False
    if B == "CS212" and b_prof != "Plantinga":
        return False
                        
    if A == "CS112" and a_prof != "Adams":
        return False
    if B == "CS112" and b_prof != "Adams":
        return False
    
    if A == "CS214" and a_prof != "Adams":
        return False
    if B == "CS214" and b_prof != "Adams":
        return False

    if A == "CS108" and a_prof != "Norman":
        return False
    if B == "CS108" and b_prof != "Norman":
        return False

    if A == "CS262" and a_prof != "Norman":
        return False
    if B == "CS262" and b_prof != "Norman":
        return False
    
    if A == "CS344" and a_prof != "VanderLinden":
        return False
    if B == "CS344" and b_prof != "VanderLinden":
        return False
    #if none of the constraints fail then return true
    return True

#Run problem
problem = CSP(variables, domains, neighbors, class_constraints)
solution = backtracking_search(problem)
print(solution)
solution = min_conflicts(problem)
print(solution)

    

{'CS108': 'Norman 8AM NH064', 'CS112': 'Adams 8AM SB382', 'CS212': 'Plantinga 9AM NH064', 'CS214': 'Adams 10AM NH064', 'CS262': 'Norman 9AM SB382', 'CS344': 'VanderLinden 10AM SB382'}
{'CS108': 'Norman 9AM NH064', 'CS112': 'Adams 11AM SB382', 'CS212': 'Plantinga 8AM SB382', 'CS214': 'Adams 10AM NH064', 'CS262': 'Norman 9AM SB382', 'CS344': 'VanderLinden 8AM NH064'}
